# SAT Solver Assignment
Group 69<br>
**Knowledge Representation**<br>
November 2024

****

## Preperations

*Everything we need to have before we can build the SAT Solver.*

**Imports** - Use this cell for all imports.

In [101]:
# import modules
import math
from tqdm import tqdm

**Data** - Import the needed data. The data needs to be in DIMACS format. Each clause in DIMACS CNF consists of:

- A series of positive or negative literals (numbers) that represent a Boolean variable or its negation.
- Each clause ends with 0.
- Each variable in DIMACS CNF is a positive integer, with unique integers representing each Boolean variable.

For Sudoku, we use a triplet `(r, c, n)`:
- `r` for the row (1 to 9)
- `c` for the column (1 to 9)
- `n` for the number (1 to 9)

In [102]:
# Geeft een file met 91 sudoku's terug, sudokus[0] is de eerste sudoku met len(sudokus[0]) == 81 (9x9=81 karakters)
with open('./top91.sdk.txt') as f:
    sudokus = f.read().splitlines()

*You are free to choose any programming language you fancy, but we must be able to run your SAT solver with the command SAT -Sn inputfile , for example: SAT -S2 sudoku_nr_10 , where SAT is the (compulsory) name of your program, n=1 for the basic DP and n=2 or 3 for your two other strategies, and the input file is the concatenation of all required input clauses (in your case: sudoku rules + given puzzle).*

****

## Part 1 - Building the SAT Solver

**The Rules of Sudoku**
1. Each number (1-9) must appear exactly once in each row.
2. Each number must appear exactly once in each column.
3. Each number must appear exactly once in each 3x3 subgrid.

**The Data of Sudoku in DIMACS**

In [103]:
def load_rules(filename: str) -> list[list[int]]:
    with open(filename) as f:
        raw_rules = f.read().splitlines()

    rules = [list(map(int, clause.split()))[:-1] for clause in raw_rules[1:]]
    return rules

def load_sudoku(filename: str) -> list[list[list[int | str]]]:
    with open(filename) as f:
        sudoku = f.read().splitlines()

    puzzles: list[list[list[int | str]]] = []
    
    puzzledim: int = math.sqrt(len(sudoku[0]))
    assert puzzledim.is_integer(), "The sudoku is not square"
    puzzledim = int(puzzledim)

    for puzzle in sudoku:
        newpuzzle = []
        for r in range(puzzledim):
            row = []
            for c in range(puzzledim):
                val = puzzle[r * puzzledim + c]
                val = int(val) if val != '.' else 0 # convert to int if we know the value
                row.append(val)
            newpuzzle.append(row)
        puzzles.append(newpuzzle)
    return puzzles

**Functions to Assign Obvious Values** - Fill in the single and pure literal clauses.

In [104]:
def single_literal(clauses: list[list[int]], assignment: dict[int, bool]) -> list[list[int]] | None:
    while any(len(clause) == 1 for clause in clauses):
        new_clauses = []
        for clause in clauses:
            if len(clause) != 1:
                new_clauses.append(clause)
                continue
            
            literal = clause[0]
            assignment[abs(literal)] = literal > 0
            # Remove satisfied clauses and update remaining clauses
            for c in clauses:
                if literal in c:
                    continue
                updated_clause = [l for l in c if l != -literal]
                if not updated_clause:
                    return None  # conflict (empty clause found)
                new_clauses.append(updated_clause)
            clauses = new_clauses
            break
    return clauses

def pure_literal(clauses: list[list[int]], assignment: dict[int, bool]) -> list[list[int]]:
    while True:
        literals = {lit for clause in clauses for lit in clause}
        pure_literals = [l for l in literals if -l not in literals]
        if not pure_literals:
            break
        for literal in pure_literals:
            assignment[abs(literal)] = literal > 0
            # Remove all clauses containing this pure literal
            clauses = [clause for clause in clauses if literal not in clause]
    return clauses

**DPLL Algorithm** - DPLL recursive algorithm function,  without heuristics, to perform the fucntions above and recursively gives values to literals.

In [105]:
def print_ass(assignment: dict) -> None:
    for key in assignment.keys():
        dimac = key if assignment[key] else -key
        print(dimac, end=' ')
    print('')

def dpll(clauses: list[list[int]], assignment: dict[int, bool], pbar, history: list[int] = [0]) -> dict[int, bool] | None:
    print("Start DPLL", history)
    print(print_ass(assignment))
    
    clauses = single_literal(clauses, assignment)
    if clauses is None:
        return None  # conflict
    if not clauses:
        return assignment  # all clauses satisfied

    clauses = pure_literal(clauses, assignment)
    if not clauses:
        return assignment  # all clauses satisfied

    # Select an unassigned literal
    literal = next((lit for clause in clauses for lit in clause if abs(lit) not in assignment), None)
    if literal is None:
        return assignment  # assignment is complete

    # Update progress bar
    pbar.update(1)

    # Check both true and false assignments
    for booly in [True, False]:
        updated_assignment = assignment.copy()
        updated_assignment[abs(literal)] = booly
        # Generate new clauses based on the current choice
        updated_clauses = [c for c in clauses if (literal if booly else -literal) not in c]
        
        result = dpll(updated_clauses, updated_assignment, pbar, history + [literal if booly else -literal])
        if result:  # If a satisfying assignment is found
            return result

    return None

*give (2)+(3) as input to (1) and return the solution to the given puzzle. This output should again be a DIMACS file, but containing only the truth assignment to all variables (729 for Sudoku, different for other SAT problems). If your input file is called 'filename', then make sure your outputfile is called 'filename.out'. If there is no solution (inconsistent problem), the output can be an empty file. If there are multiple solutions (eg. non-proper Sudoku) you only need to return a single solution.*

**Heuristics** - Two different heuristics of our choice

In [106]:
# heuristic one

In [107]:
# heuristic two

**SAT Solver** - Solves the Sudoku

Extra information about the code:
Each cell in a Sudoku puzzle can be represented by a unique variable based on its row, column, and possible number. The encoding formula for each variable is:

`variable=(i+1)×100+(j+1)×10+value`<br>

where:<br>

`i is the row index (0 to 8)`<br>
`j is the column index (0 to 8)`<br>
`value is the number in the cell (1 to 9)`<br>

This formula converts each cell and its possible values into unique numbers by combining the row, column, and value in a single integer. For example, if a cell at row 0, column 0 can be a 5, the encoded variable would be:<br>

`variable=(0+1)×100+(0+1)×10+5=115`<br>

In [108]:
def solve_sudoku(rules: list[list[int]], puzzle: list[list[int]], size: int) -> list[list[int]] | None:
    """
    Solves a Sudoku puzzle using given rules and an initial puzzle setup.

    Args:
        rules (list[list[int]]): A list of rules in CNF form for Sudoku constraints.
        puzzle (list[list[int]]): A 9x9 grid representing the puzzle, with `0` for empty cells.
        size (int): The size of the Sudoku puzzle (e.g. 9 for a 9x9 puzzle).

    Returns:
        list[list[int]] | None: A solved 9x9 Sudoku grid if a solution exists, otherwise None.
    """
    assignment = {} # the sudoku
    amt_unknown = 0
    for i, row in enumerate(puzzle):
        for j, value in enumerate(row):
            if value != 0:  # if 0, cell is empty, if not 0, it is a pre-filled cell
                # Make the value in row i, column j True if it is the value from the sudoku, all other numbers in that place False
                assignment[(i + 1) * 100 + (j + 1) * 10 + value] = True
                for k in range(1, size+1):
                    if k != value:
                        assignment[(i + 1) * 100 + (j + 1) * 10 + k] = False
                    
            else:
                amt_unknown += 1
    print(assignment)
    with tqdm(total=amt_unknown * size,desc='Trying to find logic solution') as pbar:
        solution = dpll(rules, assignment, pbar)
    if not solution:
        return None
    
    # parse the solution back to a 9x9 grid
    result = [[0] * 9 for _ in range(9)]
    for var, value in solution.items():
        if value:
            i = (var // 100) - 1
            j = ((var // 10) % 10) - 1
            num = var % 10
            result[i][j] = num
    return result

**Test Sudoku** - Try and see if it works

In [109]:
load_sudoku('./4x4_easy.txt')[1]

[[1, 2, 3, 4], [3, 4, 1, 0], [2, 1, 4, 3], [4, 3, 0, 1]]

In [110]:
# Load the Sudoku rules and the puzzle
filename = './sudoku-rules-9x9.txt'
rules = load_rules(filename)

# Load the puzzles from the file and select the second puzzle (index 1)
puzzle = load_sudoku('./veryeasy.txt')[0]
rightpuzzle = load_sudoku('./veryeasy.txt')[-1]
print(puzzle)

size = 9  # Size of the grid (4x4 Sudoku)

# Solve the Sudoku puzzle
solution = solve_sudoku(rules, puzzle, size)

print('puzzle:     ', puzzle)
print('solution:   ', solution)
print('rightpuzzle:', rightpuzzle)
print(solution == rightpuzzle)
# If a solution is found, print the solved Sudoku grid
if solution:
    for row in solution:
        print(row)
        if 0 in row:
            print("PANIEK")
else:
    print("No solution found")
    # 263 385 517 566 647

[[9, 8, 4, 5, 2, 7, 3, 1, 6], [5, 2, 7, 1, 6, 3, 8, 4, 9], [6, 1, 3, 8, 9, 4, 7, 5, 2], [1, 6, 2, 9, 4, 8, 5, 3, 7], [7, 5, 8, 3, 1, 6, 2, 9, 4], [3, 4, 9, 7, 5, 2, 6, 8, 1], [0, 9, 1, 2, 7, 5, 4, 6, 3], [4, 7, 5, 6, 3, 9, 1, 2, 8], [2, 3, 6, 4, 8, 1, 9, 7, 5]]
{119: True, 111: False, 112: False, 113: False, 114: False, 115: False, 116: False, 117: False, 118: False, 128: True, 121: False, 122: False, 123: False, 124: False, 125: False, 126: False, 127: False, 129: False, 134: True, 131: False, 132: False, 133: False, 135: False, 136: False, 137: False, 138: False, 139: False, 145: True, 141: False, 142: False, 143: False, 144: False, 146: False, 147: False, 148: False, 149: False, 152: True, 151: False, 153: False, 154: False, 155: False, 156: False, 157: False, 158: False, 159: False, 167: True, 161: False, 162: False, 163: False, 164: False, 165: False, 166: False, 168: False, 169: False, 173: True, 171: False, 172: False, 174: False, 175: False, 176: False, 177: False, 178: False, 

Trying to find logic solution:   0%|          | 0/9 [00:00<?, ?it/s]

Trying to find logic solution: 100%|██████████| 9/9 [00:00<00:00, 87.71it/s]

Start DPLL [0]
119 -111 -112 -113 -114 -115 -116 -117 -118 128 -121 -122 -123 -124 -125 -126 -127 -129 134 -131 -132 -133 -135 -136 -137 -138 -139 145 -141 -142 -143 -144 -146 -147 -148 -149 152 -151 -153 -154 -155 -156 -157 -158 -159 167 -161 -162 -163 -164 -165 -166 -168 -169 173 -171 -172 -174 -175 -176 -177 -178 -179 181 -182 -183 -184 -185 -186 -187 -188 -189 196 -191 -192 -193 -194 -195 -197 -198 -199 215 -211 -212 -213 -214 -216 -217 -218 -219 222 -221 -223 -224 -225 -226 -227 -228 -229 237 -231 -232 -233 -234 -235 -236 -238 -239 241 -242 -243 -244 -245 -246 -247 -248 -249 256 -251 -252 -253 -254 -255 -257 -258 -259 263 -261 -262 -264 -265 -266 -267 -268 -269 278 -271 -272 -273 -274 -275 -276 -277 -279 284 -281 -282 -283 -285 -286 -287 -288 -289 299 -291 -292 -293 -294 -295 -296 -297 -298 316 -311 -312 -313 -314 -315 -317 -318 -319 321 -322 -323 -324 -325 -326 -327 -328 -329 333 -331 -332 -334 -335 -336 -337 -338 -339 348 -341 -342 -343 -344 -345 -346 -347 -349 359 -351 -352 -35